### Looking for global fit (standard GLM-HMM) for Psytrack animals all together

In [1]:
# importing packages and modules
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import minimize
from utils import *
from plotting_utils import *
from analysis_utils import *
import dglm_hmm1
from scipy.stats import multivariate_normal, norm
# import jax

# code from summer project working with PWM data
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', '..', 'LC_PWM_GLM-HMM/code')))
import io_utils, analysis_utils, plotting_utils

sns.set_context("talk")

In [2]:
animalsIDs = ['W060','W068','W073','W074','W075','W080','W081','W082','W088','W089']
pni_path = '/Users/lencacuturela/Desktop/github/LC_PWM_GLM-HMM/data'

In [3]:
# According to Jonathan's suggestions for fitting global GLM-HMM for all animals
# stage filter changed from 3 to 4
# cutoff changed from 10 to 50
firstAnimal = 'W060'
x, y = io_utils.prepare_design_matrices(rat_id=firstAnimal, path=pni_path, psychometric=True, cutoff=100, stage_filter=4, overwrite=False)
sessInd = list(io_utils.session_start(rat_id=firstAnimal, path=pni_path, psychometric=True, cutoff=100, stage_filter=4))

for animal in animalsIDs:
    if (animal != firstAnimal):
        xTemp, yTemp = io_utils.prepare_design_matrices(rat_id=animal, path=pni_path, psychometric=True, cutoff=100, stage_filter=4, overwrite=False)
        sessIndTemp = list(io_utils.session_start(rat_id=animal, path=pni_path, psychometric=True, cutoff=100, stage_filter=4))
        x = np.concatenate((x,xTemp))
        y = np.concatenate((y,yTemp))
oneSessInd = [0,x.shape[0]]

# NOT TAKING INTO ACCOUNT TO BREAK FORWARD-BACKWARD AT EACH SESSION LIKE IRIS DOES
    

In [6]:
inits = 30
penaltyW=True
maxiter = 250

path = pni_path

N = x.shape[0]
D = x.shape[1]
C = 2 # only looking at binomial classes
K = 2

allP = np.zeros((inits, K, K))
allW = np.zeros((inits, N, K, D, C))
allLl = np.zeros((inits, maxiter))
testLl = np.zeros((inits))
oneSessInd = [0,N]
dGLM_HMM = dglm_hmm1.dGLM_HMM1(N,K,D,C)

for i in range(0,inits):
    print("Init ", i)

    # global fit
    if (i%2 ==0):
        initP0, initW0 = dGLM_HMM.generate_param(sessInd=oneSessInd, transitionDistribution=['dirichlet', (5, 1)], weightDistribution=['uniform', (-2,2)]) 
    else:
        initP0, initW0 = dGLM_HMM.generate_param(sessInd=oneSessInd, transitionDistribution=['dirichlet', (5, 1)], weightDistribution=['normal', (0,2)]) 
    allP[i], allW[i], allLl[i] = dGLM_HMM.fit(x, y,  initP0, initW0, sigma=reshapeSigma(0, K, D), sessInd=oneSessInd, pi0=None, maxIter=maxiter, tol=1e-3, penaltyW=penaltyW) # sigma does not matter here
    
    np.save(f'../data_PWM/trainLl-global_PWM_all-animals_sf=4_{K}_state_multiple_inits_penaltyW={penaltyW}', allLl)
    np.save(f'../data_PWM/P-global_PWM_all-animals_sf=4_{K}_state_multiple_inits_penaltyW={penaltyW}', allP)
    np.save(f'../data_PWM/W-global_PWM_all-animals_sf=4_{K}_state_multiple_inits_penaltyW={penaltyW}', allW)

    

Init  0
0
